<a href="https://colab.research.google.com/github/nathaliemondoukpe12/app_gest_task/blob/main/EducationM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/content/sample_data/education_dataset.csv')

In [ ]:
# Afficher les 5 premières lignes pour vérification
print("--- Tête du DataFrame ---")
print(data.head())

--- Tête du DataFrame ---
   student_id  age gender  hours_study_per_week  attendance_rate  \
0           1   15      M                    16             68.6   
1           2   14      F                     1             92.2   
2           3   16      F                    13             72.6   
3           4   17      M                    16             80.8   
4           5   14      F                     6             96.0   

  parent_education_level school_type  math_score  reading_score final_result  
0                 medium      public          61             96         pass  
1                 medium     private          67             95         fail  
2                 medium      public          80             31         pass  
3                   high      public          21             82         fail  
4                 medium      public           2             49         pass  


In [ ]:
# Identifier les colonnes numériques à standardiser
features_to_scale = [
    'age',
    'hours_study_per_week',
    'attendance_rate',
    'math_score',
    'reading_score'
]
# Standardisation des données
scaler = StandardScaler()
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

In [ ]:
# Afficher les statistiques pour confirmer la standardisation (moyenne proche de 0, écart-type proche de 1)
print("\n--- Statistiques après Standardisation ---")
print(data[features_to_scale].describe().T)


--- Statistiques après Standardisation ---
                      count          mean       std       min       25%  \
age                    50.0  2.597922e-16  1.010153 -1.572055 -0.948224   
hours_study_per_week   50.0 -7.549517e-17  1.010153 -1.598194 -0.848489   
attendance_rate        50.0 -5.151435e-16  1.010153 -1.856738 -0.738032   
math_score             50.0 -1.537659e-16  1.010153 -1.812102 -0.785045   
reading_score          50.0  6.661338e-17  1.010153 -1.976703 -0.733069   

                           50%       75%       max  
age                  -0.012477  0.923270  1.547102  
hours_study_per_week -0.098785  0.695020  1.577026  
attendance_rate       0.231048  0.846945  1.595069  
math_score            0.073948  0.830235  1.885303  
reading_score         0.086380  0.886547  1.802401  


Construction et Entraînement du Modèle


In [ ]:
data['final_result_encoded'] = data['final_result'].map({'pass': 1, 'fail': 0})
# Définition des variables
X = data[features_to_scale]
y = data['final_result_encoded'] # La variable cible encodée (Réussite/Échec)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintient la proportion de réussite/échec
)

# Initialisation et entraînement du modèle de Régression Logistique
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"\n--- Évaluation du Modèle ---")
print(f"Précision (Accuracy): {accuracy:.4f}")
print("\nRapport de Classification:\n", classification_report(y_test, y_pred))


--- Évaluation du Modèle ---
Précision (Accuracy): 0.5000

Rapport de Classification:
               precision    recall  f1-score   support

           0       0.50      0.20      0.29         5
           1       0.50      0.80      0.62         5

    accuracy                           0.50        10
   macro avg       0.50      0.50      0.45        10
weighted avg       0.50      0.50      0.45        10



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("\n--- Analyse des Coefficients de Régression Logistique ---")
coefficients = pd.Series(model.coef_[0], index=X.columns)
coefficients_sorted = coefficients.sort_values(ascending=False)
print(coefficients_sorted)

# Comparaison des coefficients (Intuition vs Réalité)
print("\n--- Comparaison Intuition vs. Réalité ---")
for feature, coef in coefficients_sorted.items():
    signe = "POSITIVE" if coef > 0 else "NÉGATIVE"
    force = "FORTE" if abs(coef) > 0.5 else "FAIBLE"

    print(f"Facteur '{feature}': Coef = {coef:.3f} ({force} {signe})")


--- Analyse des Coefficients de Régression Logistique ---
math_score              0.177542
age                     0.098167
hours_study_per_week   -0.058608
reading_score          -0.253868
attendance_rate        -0.353862
dtype: float64

--- Comparaison Intuition vs. Réalité ---
Facteur 'math_score': Coef = 0.178 (FAIBLE POSITIVE)
Facteur 'age': Coef = 0.098 (FAIBLE POSITIVE)
Facteur 'hours_study_per_week': Coef = -0.059 (FAIBLE NÉGATIVE)
Facteur 'reading_score': Coef = -0.254 (FAIBLE NÉGATIVE)
Facteur 'attendance_rate': Coef = -0.354 (FAIBLE NÉGATIVE)
